<a href="https://colab.research.google.com/github/Arihant987/LLM/blob/main/Train_Small_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Training the small model{distil gpt2} on very small training data")

Training the small model{distil gpt2} on very small training data


In [2]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
# ast is abstract syntax tree
import datasets
from tqdm import tqdm
import time

In [4]:
## Load data
data_sample=load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [5]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [6]:
updated_data=[{'Name':item['Name'],'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [7]:
df=pd.DataFrame(updated_data)

In [8]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [9]:
# extract the symptoms just to maintain uniformity
df['Symptoms']=df['Symptoms'].apply(lambda x:', '.join(x.split(',')))

In [10]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
  device=torch.device('cpu')

In [12]:
device

device(type='cuda')

In [13]:
tokenizer=GPT2Tokenizer.from_pretrained('distilgpt2')
model=GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
batch_size=8

In [15]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [16]:
# Dataset preparation

class LanuageDataset(Dataset):
  def __init__(self,data,tokenizer):
    self.labels=df.columns
    self.data=df.to_dict(orient='records')
    self.tokenizer=tokenizer
    x=self.fittest_max_length(df)
    self.max_length=x

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    x=self.data[idx][self.labels[0]]
    y=self.data[idx][self.labels[1]]
    text=f"{x}|{y}"
    # pt is pytorch tensors, truncation is if len(tokens)>128 it will keep first 128 tokens
    # By default, encode_plus adds special tokens like [CLS] (for classification tasks) and [SEP] (to separate sequences).
    tokens=self.tokenizer.encode_plus(text,return_tensors='pt',max_length=128,padding='max_length',truncation=True)
    return tokens

  def fittest_max_length(self,df):
    '''
    Smallest power of 2 larger than longest term in the dataset,
    to speed up training time as most gpu are optimized for power of 2 operations
    '''
    max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
    x=2
    while x<max_length:
      x*=2
    return x


data_sample=LanuageDataset(df,tokenizer)

In [17]:
data_sample

In [18]:
train_size=int(0.8*len(data_sample))
valid_size=len(data_sample)-train_size

train_data,valid_data=random_split(data_sample,[train_size,valid_size])

In [19]:
# Make the iterators

# shuffle batches for new variety and diversity
train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
valid_loader=DataLoader(valid_data,batch_size=batch_size)


In [20]:
batch_size=batch_size
num_epochs=5
model_name='distilgpt2'
gpu=0

In [21]:
criterion=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer=optim.Adam(model.parameters(),lr=5e-4)
tokenizer.pad_token=tokenizer.eos_token

In [22]:
results = pd.DataFrame(columns=['epoch','transformer','batch_size','gpu',
                                'training_loss','validation_loss','epoch_duration_sec'])


In [23]:
## Training the loop

for epoch in range(num_epochs):
  start_time=time.time()

  ## TRAINING

  model.train() # This line tells the model we're in 'learning mode'
  epoch_train_loss=0
  train_iterator=tqdm(train_loader,desc=f'Training Epoch  {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}')
  for batch in train_iterator:
    optimizer.zero_grad()
    # Removes dimensions of size 1 from the tensor
    inputs=batch['input_ids'].squeeze(1).to(device)
    # Appropriate Cloning for MLM(Masked Language Modeling):
    targets=inputs.clone()
    outputs=model(input_ids=inputs,labels=targets)
    loss=outputs.loss
    loss.backward()
    optimizer.step()
    train_iterator.set_postfix({'Training Loss':loss.item()})
    epoch_train_loss+=loss.item()
  avg_epoch_training_loss=epoch_train_loss/len(train_iterator)

  ## VALIDATION


  ## This line below tells the model to 'stop learning'
  model.eval()
  epoch_validation_loss=0
  total_loss=0
  valid_iterator=tqdm(valid_loader,desc=f'Validation Epoch  {epoch+1}/{num_epochs}')
  with torch.no_grad():
    for batch in valid_iterator:
      inputs=batch['input_ids'].squeeze(1).to(device)
      targets=inputs.clone()
      outputs=model(input_ids=inputs,labels=targets)
      loss=outputs.loss
      total_loss+=loss
      valid_iterator.set_postfix({'Validation Loss':loss.item()})
      epoch_validation_loss+=loss.item()

    avg_epoch_validation_loss=epoch_validation_loss/len(valid_loader)

    end_time=time.time()
    epoch_duration_sec=end_time-start_time

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}

  results.loc[len(results)]=new_row
  # access and modify the data
  print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")

Training Epoch  1/5 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:09<00:00,  4.24it/s, Training Loss=0.766]
Validation Epoch  1/5: 100%|██████████| 10/10 [00:00<00:00, 16.14it/s, Validation Loss=0.645]


Epoch: 1, Validation Loss: 0.6026811599731445


Training Epoch  2/5 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.97it/s, Training Loss=0.521]
Validation Epoch  2/5: 100%|██████████| 10/10 [00:00<00:00, 15.78it/s, Validation Loss=0.621]


Epoch: 2, Validation Loss: 0.5639762282371521


Training Epoch  3/5 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.85it/s, Training Loss=0.304]
Validation Epoch  3/5: 100%|██████████| 10/10 [00:00<00:00, 16.49it/s, Validation Loss=0.633]


Epoch: 3, Validation Loss: 0.5670876502990723


Training Epoch  4/5 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.79it/s, Training Loss=0.348]
Validation Epoch  4/5: 100%|██████████| 10/10 [00:00<00:00, 16.29it/s, Validation Loss=0.655]


Epoch: 4, Validation Loss: 0.5865957140922546


Training Epoch  5/5 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.87it/s, Training Loss=0.279]
Validation Epoch  5/5: 100%|██████████| 10/10 [00:00<00:00, 17.45it/s, Validation Loss=0.674]

Epoch: 5, Validation Loss: 0.6194902658462524


In [ ]:
input_str="Kidney Failure"
input_ids=tokenizer.encode(input_str,return_tensors='pt').to(device)

output=model.generate(input_ids,
                      max_length=25,
                      # How many different sequences model will generate.
                      num_return_sequences=1,
                      do_sample=True,
                      top_k=8,
                      # Nucleus sampling only tokens whose cumulative probability adds up to 95% will be considered for sampling
                      top_p=0.95,
                      temperature=0.5,
                      repetition_penalty=1.1)

decoded_output=tokenizer.decode(output[0],skip_special_tokens=True)

In [33]:
print(decoded_output)

Kidney Failure|Decreased urine output,  swelling of the legs or ankles,  nausea or vomiting


In [31]:
torch.save(model, 'SmallLMTrain.pt')

In [ ]:
# Sampling involves choosing the next word based on the probability distribution over the vocabulary that a neural network produces